# LOC/issue stats by year

In [10]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import csv
import datetime
import pytz

## Compute line of code additions/deletions/totals segmented by year, repo

In [ ]:
cols = ['author_date', 'author', 'commit_sha', 'additions', 'deletions', 'total', 'files_changed', 'message', 'repo']
# commit_df1 = pd.read_csv("../commit_stats.csv", index_col='author_date', usecols=cols)
# commit_df2 = pd.read_csv("../commit_stats2.csv", index_col='author_date', usecols=cols)

# commit_df = pd.concat([commit_df1, commit_df2])

# after duplicate drop
commit_df = pd.read_csv('../all_commit_stats_2021-2025.csv', index_col='author_date', usecols=cols)
commit_df.index = pd.to_datetime(commit_df.index)

In [41]:
len(commit_df)

37469

In [161]:
stats_df = pd.DataFrame()

for year in [2021, 2022, 2023, 2024, 2025]:
    start_time = pytz.utc.localize(datetime.datetime(year, 1, 1))
    end_time = pytz.utc.localize(datetime.datetime(year, 12, 31))

    filt_df = commit_df[
        (commit_df.index >= start_time) & (commit_df.index <= end_time)
    ][['repo', 'additions', 'deletions', 'total', 'files_changed']]

    grouped = filt_df.groupby(by=['repo']).sum()
    grouped = grouped.join(filt_df['repo'].value_counts(), on='repo')
    grouped.rename(columns={"count": "n_commits"}, inplace=True)
    
    # add year to make indexing easier
    grouped = grouped.T
    grouped['year'] = year

    stats_df = pd.concat([stats_df, grouped])

# quick cleanup of naming etc
stats_df = stats_df.reset_index().set_index(['year', 'index'])
stats_df.index = stats_df.index.set_names(['year', 'stat'])
stats_df.columns.name = ''

In [163]:
stats_df.T

year                                2021                                   \
stat                           additions deletions    total files_changed   
                                                                            
Galil                             1722.0    1403.0   3125.0          63.0   
MEC_RIS_PAC                          1.0       0.0      1.0           1.0   
PCB-Laser-Environmental-Sensor       0.0       0.0      0.0           2.0   
ads-async                        12097.0    5322.0  17419.0         225.0   
ads-deploy                         120.0      34.0    154.0          25.0   
...                                  ...       ...      ...           ...   
ioc-xcs-pem                          NaN       NaN      NaN           NaN   
ioc-xcs-xrt-pem                      NaN       NaN      NaN           NaN   
lcls-plc-mec-be-lens-interlock       NaN       NaN      NaN           NaN   
lcls-scan-gui                        NaN       NaN      NaN           NaN   
pcds-ci-test-internal                NaN       NaN      NaN           NaN   

year                                          2022                    \
stat                           n_commits additions deletions   total   
                                                                       
Galil                                7.0    2532.0    2260.0  4792.0   
MEC_RIS_PAC                          1.0       NaN       NaN     NaN   
PCB-Laser-Environmental-Sensor       2.0       NaN       NaN     NaN   
ads-async                           80.0    2146.0     492.0  2638.0   
ads-deploy                          10.0     642.0     686.0  1328.0   
...                                  ...       ...       ...     ...   
ioc-xcs-pem                          NaN       NaN       NaN     NaN   
ioc-xcs-xrt-pem                      NaN       NaN       NaN     NaN   
lcls-plc-mec-be-lens-interlock       NaN       NaN       NaN     NaN   
lcls-scan-gui                        NaN       NaN       NaN     NaN   
pcds-ci-test-internal                NaN       NaN       NaN     NaN   

year                                                    ...      2024  \
stat                           files_changed n_commits  ... additions   
                                                        ...             
Galil                                   37.0      14.0  ...   91346.0   
MEC_RIS_PAC                              NaN       NaN  ...       NaN   
PCB-Laser-Environmental-Sensor           NaN       NaN  ...       NaN   
ads-async                               53.0      16.0  ...       NaN   
ads-deploy                              57.0      17.0  ...       NaN   
...                                      ...       ...  ...       ...   
ioc-xcs-pem                              NaN       NaN  ...       NaN   
ioc-xcs-xrt-pem                          NaN       NaN  ...       NaN   
lcls-plc-mec-be-lens-interlock           NaN       NaN  ...       NaN   
lcls-scan-gui                            NaN       NaN  ...       NaN   
pcds-ci-test-internal                    NaN       NaN  ...       NaN   

year                                                                        \
stat                           deletions     total files_changed n_commits   
                                                                             
Galil                            90835.0  182181.0         351.0       8.0   
MEC_RIS_PAC                          NaN       NaN           NaN       NaN   
PCB-Laser-Environmental-Sensor       NaN       NaN           NaN       NaN   
ads-async                            NaN       NaN           NaN       NaN   
ads-deploy                           NaN       NaN           NaN       NaN   
...                                  ...       ...           ...       ...   
ioc-xcs-pem                          NaN       NaN           NaN       NaN   
ioc-xcs-xrt-pem                      NaN       NaN           NaN       NaN   
lcls-plc-mec-be-lens-interlock 

In [207]:
stats_df.T.to_csv('../grouped_loc_data.csv')

# Issue stats aggregation 

In [105]:
idf = pd.read_csv('../issue_stats_250218.csv')
idf.created_at = pd.to_datetime(idf.created_at)
idf.closed_at = pd.to_datetime(idf.closed_at)

In [106]:
idf.head()

,Unnamed: 0,number,state,created_at,closed_at,title,is_pull,repository
0,0,356,open,2025-02-12 22:52:58+00:00,NaT,First pass at allowing for cached devices to b...,True,happi
1,1,355,closed,2025-02-11 17:45:21+00:00,2025-02-11 20:32:32+00:00,"Fix minor typos in readme, add generated _vers...",True,happi
2,2,354,open,2025-01-31 18:59:31+00:00,NaT,happi.cli.search_parser has misleading annotat...,False,happi
3,3,353,closed,2025-01-23 19:34:57+00:00,2025-02-10 19:43:58+00:00,BLD: split ui dependencies into subpackage,True,happi
4,4,352,closed,2025-01-09 16:40:58+00:00,2025-01-09 20:59:02+00:00,"BLD: Try swapping to line_profiler, seems the ...",True,happi


In [203]:

iss_df = pd.DataFrame()

for year in [2021, 2022, 2023, 2024, 2025]:
    start_time = pytz.utc.localize(datetime.datetime(year, 1, 1))
    end_time = pytz.utc.localize(datetime.datetime(year, 12, 31))
    summ = {'repo': [], 'issues_created': [], 'issues_closed': [],
            'pulls_created': [], 'pulls_closed': []}
    for repo in idf.repository.unique():
        summ['repo'].append(repo)

        issues_created = len(idf[
            (idf['created_at'] >= start_time)
            & (idf['created_at'] <= end_time)
            & (idf['repository'] == repo)
            & ~idf['is_pull']
        ])
        summ['issues_created'].append(issues_created)

        issues_closed = len(idf[
            (idf['closed_at'] >= start_time)
            & (idf['closed_at'] <= end_time)
            & (idf['repository'] == repo)
            & ~idf['is_pull']
        ])
        summ['issues_closed'].append(issues_closed)
        
        pulls_created = len(idf[
            (idf['created_at'] >= start_time)
            & (idf['created_at'] <= end_time)
            & (idf['repository'] == repo)
            & idf['is_pull']
        ])
        summ['pulls_created'].append(pulls_created)
        
        pulls_closed = len(idf[
            (idf['closed_at'] >= start_time)
            & (idf['closed_at'] <= end_time)
            & (idf['repository'] == repo)
            & idf['is_pull']
        ])
        summ['pulls_closed'].append(pulls_closed)

    year_df = pd.DataFrame(data=summ, columns=summ.keys())
    year_df = year_df.set_index('repo')

    year_df = year_df.T
    year_df['year'] = year

    iss_df = pd.concat([iss_df, year_df])

iss_df = iss_df.reset_index().set_index(['year','index'])


In [206]:
iss_df.T.to_csv('../grouped_issue_data.csv')